In [2]:
import pandas as pd
import json
import requests
from time import sleep

/Users/dushamishkin/VSCProjects/Thesis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [1]:
api_key   = "no leaking!"

In [3]:
def make_request(unloco, fromdate, todate):
    url = f"https://api.myshiptracking.com/api/v2/port/calls?unloco={unloco}&fromdate={fromdate}&todate={todate}"

    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    print(f"Making request to {url}")
    response = requests.get(url, headers=headers)
    if not response.ok:
        print("Error:", response.status_code, response.text)

    return response.json()

In [36]:
baseline_df = pd.DataFrame()

In [37]:
def to_df(json_data):
    data = json_data.get("data", [])
    new_df = pd.DataFrame(data)

    return new_df

In [38]:
def get_port_data(unloco):
    combined_df = pd.DataFrame()
    
    for month in range(1, 13):
        month_str = f"{month:02d}"
        fromdate = f"2024-{month_str}-01T00:00:00Z"
        todate = f"2024-{month_str}-31T00:00:00Z"

        data = make_request(unloco, fromdate, todate)
        if data:
            new_df = to_df(data)
            combined_df = pd.concat([combined_df, new_df], ignore_index=True)
            # print(f"Port {unloco}: data for {month_str} added.")
        else:
            print(f"Port {unloco}: no data for {month_str}.")
        
        sleep(1)
    
    return combined_df

In [39]:
ports_dict = {
    "toamasina": "MGTOA",
    "durban": "ZADUR",
    "richards_bay": "ZARCB",
    "cape_town": "ZACPT",
    "port_elizabeth": "ZAPLZ",
    "east_london": "ZAELS",
    "saldanha": "ZASDB",
    "coega": "ZAZBA",
    "walvis_bay": "NAWVB",
    "beira": "MZBEW",
    "maputo": "MZMPM",
    "djibouti": "DJJIB",
    "said": "EGPSD",
    "mombasa": "KEMBA",
    "casablanca": "MACAS",
    "lome": "TGLFW",
    "abidjan": "CIABJ",
    "tema": "GHTEM",
    "dakar": "SNDKR",
    "douala": "CMDLA",
    "gentil": "GAPOG",
    "conakry": "GNCKY",
    "pointe_noire": "CGPNR",
    "matadi": "CDMAT",
    "monrovia": "LRMLW",
    "cotonou": "BJCOO",
    "dar_es_salaam": "TZDAR",
    "berbera": "SOBBO"
}

In [40]:
baseline_df = pd.DataFrame().to_csv("data/baseline.csv", index=False)

In [41]:
for unloco in ports_dict.values():
    print(f"PROCESSING PORT: {unloco}")
    port_data = get_port_data(unloco)
    if not port_data.empty:
        baseline_df = pd.concat([baseline_df, port_data], ignore_index=True)
        baseline_df.to_csv(f"data/baseline_{unloco}.csv", index=False)
        print(f"Data for port {unloco} added to baseline DataFrame.")
        print(f"Unique port ids: {baseline_df['port_name'].unique()}\n")
    else:
        print(f"No data for port {unloco}.")
    
    _ = input("Press Enter to continue")

PROCESSING PORT: MGTOA
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-01-01T00:00:00Z&todate=2024-01-31T00:00:00Z
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-02-01T00:00:00Z&todate=2024-02-31T00:00:00Z
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-03-01T00:00:00Z&todate=2024-03-31T00:00:00Z
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-04-01T00:00:00Z&todate=2024-04-31T00:00:00Z
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-05-01T00:00:00Z&todate=2024-05-31T00:00:00Z
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-06-01T00:00:00Z&todate=2024-06-31T00:00:00Z
Making request to https://api.myshiptracking.com/api/v2/port/calls?unloco=MGTOA&fromdate=2024-07-01T00:00:00Z&todate=2024-07-31T00:00:00Z
Making requ

In [42]:
baseline_df.to_csv("data/ALL_ports.csv", index=False)